In [1]:
from tweepy import *
from datetime import datetime, date, time, timezone, timedelta
from collections import Counter
import sys
import os

In [2]:
from dotenv import load_dotenv #storing env variables in .env file
load_dotenv()

True

In [ ]:
consumer_key=os.environ.get("key")
consumer_secret=os.environ.get("secret-key")
access_token = os.environ.get("access-token")
access_token_secret = os.environ.get("access-token-secret")

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth, wait_on_rate_limit=False)

In [15]:
target = "elonmusk"
item = auth_api.get_user(screen_name="dhruv__s")
#datetime.now(timezone.utc), item.created_at
#(datetime.now(timezone.utc) - item.created_at).days
tweets = Cursor(auth_api.user_timeline, screen_name=target).items()
tws = []
rtws = []
ht = {}
for tw in tweets:
    if hasattr(tw, 'retweeted_status'):
        rtws.append((tw.text, tw.retweet_count, tw.created_at.strftime("%m/%d/%Y, %H:%M:%S")))
    else:
        tws.append((tw.text, tw.retweet_count, tw.created_at.strftime("%m/%d/%Y, %H:%M:%S")))
     
    for h in tw.entities['hashtags']:
        if h['text'] in ht:
            ht[h['text']] += 1
        else:
            ht[h['text']] = 1



tws.sort(key = lambda x: x[1])
rtws.sort(key = lambda x: x[1])
#print(sorted(ht, key=ht.get, reverse=True)[:5])
print(tws[-6:-1])

[('Sus man', 16582, '05/20/2022, 19:28:26'), ('@catturd2 They began brewing attacks of all kinds as soon as the Twitter acquisition was announced.\n\nIn my 30 year… https://t.co/H1GB2QgwtH', 16902, '05/20/2022, 12:37:56'), ('Tesla is building a hardcore litigation department where we directly initiate &amp; execute lawsuits. The team will rep… https://t.co/ElDf9wWeCV', 17216, '05/20/2022, 20:28:56'), ('@Jim_Jordan All true. \n\nBet most people still don’t know that a Clinton campaign lawyer, using campaign funds, crea… https://t.co/oXHmWxPHfb', 21113, '05/20/2022, 18:36:26'), ('Finally, we get to use Elongate as scandal name. It’s kinda perfect. 🤣 https://t.co/qSNH7lsn72', 25976, '05/20/2022, 05:43:53')]


In [22]:
x = User("rossiadam")
print(x.hashtags)

{'hokies': 37, 'covid': 35, 'bitcoin': 31, 'inflation': 25, 'virginia': 22}


In [26]:
target = "dhruv__s"

u = auth_api.get_user(target)
print("ID:", u.id_str)
print("Grabbing data from: ", u.screen_name)



TypeError: get_user() takes 1 positional argument but 2 were given

In [ ]:
def get_followers(user_name):
    followers = []
    fcall = Cursor(auth_api.get_followers, screen_name=user_name).items(20)

    for fol in fcall:
        try:
            followers.append(fol.screen_name)
        except TweepError as e:
            print(f'Waiting {60}s for limit:', e)
            time.sleep(60)

    return followers
print(get_followers("elonmusk"))

In [18]:
class User:
    
    def __init__(self, target):
        self.target = target
        
        self.item = auth_api.get_user(screen_name=target)        
        self.name = self.item.name
        self.user = self.item.screen_name
        self.desc = self.item.description
        self.follower_count = self.item.followers_count
        self.age = (datetime.now(timezone.utc) - self.item.created_at).days
        
        self.tws, self.rtws, self.hashtags = self.retweets()
        
        self.followers = self.get_followers()
        self.new_followers = self.get_new_followers()
        
    
    def retweets(self):
        tweets = Cursor(auth_api.user_timeline, screen_name=self.user).items()
        tws = []
        rtws = []
        ht = {}
        
        for tw in tweets:
            if hasattr(tw, 'retweeted_status'):
                rtws.append((tw.text, tw.retweet_count, tw.created_at.strftime("%m/%d/%Y, %H:%M:%S")))
            else:
                tws.append((tw.text, tw.retweet_count, tw.created_at.strftime("%m/%d/%Y, %H:%M:%S")))
                
            for h in tw.entities['hashtags']:
                if h['text'] in ht:
                    ht[h['text']] += 1
                else:
                    ht[h['text']] = 1
                    
        if len(ht) >= 5:
            ht = dict(Counter(ht).most_common(5))
                
        tws.sort(key = lambda x: x[1])
        rtws.sort(key = lambda x: x[1])
        
        if len(rtws) < 5 and len(tws) < 5:
            return tws, rtws, ht
        
        if len(rtws) < 5:
            return tws[-6:-1], rtws, ht
        
        if len(tws) < 5:
            return tws, rtws[-6:-1], ht
            
        return tws[-6:-1], rtws[-6:-1], ht
    
    def get_followers(self):
        followers = []
        fcall = Cursor(auth_api.get_followers, screen_name=self.user).items(50)
        
        for fol in fcall:
            try:
                followers.append(fol.screen_name)
            except TweepError as e:
                print(f'Waiting {60}s for limit:', e)
                time.sleep(60)

        return followers
    
    def get_new_followers(self):
        if len(self.followers) < 10:
            return self.followers
        return self.followers[0:10]
    
    